# Creating S3 Bucket

In [2]:
import boto3
s3 = boto3.resource("s3")

In [10]:
bucket_name = "stockpricemarketprediction"
try:
    s3.create_bucket(Bucket=bucket_name)
    print('s3 bucket has been created successfully')
except Exception as e:
    print('s3 error: ',e)

s3 bucket has been created successfully


## Creating train and validation csv

In [12]:
!pip install yfinance

import pandas as pd
from datetime import datetime
import yfinance as yf

# initialize parameters
start_date = datetime(2019, 1, 1)
end_date = datetime(2021, 1, 1)
# get the data (FOR APPLE)
df_data = yf.download('AAPL', start = start_date, end = end_date)
df_data.reset_index(inplace=True)
df_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.7 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 96.3 MB/s eta 0:00:00:00:0100:01
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-linux_x86_64.whl size=275946 sha256=57a3d8aca352242027009f3d72f66f04bf2b8738b4f1474c5ce1cbcac1d5b40a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.893322,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.118881,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.575390,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.172871,164101200
...,...,...,...,...,...,...,...
500,2020-12-24,131.320007,133.460007,131.100006,131.970001,129.679733,54930100
501,2020-12-28,133.990005,137.339996,133.509995,136.690002,134.317810,124486200
502,2020-12-29,138.050003,138.789993,134.339996,134.869995,132.529373,121047300
503,2020-12-30,135.580002,135.990005,133.399994,133.720001,131.399368,96452100


## ETL

In [13]:
df_data.drop (axis=1, columns = ['Adj Close'], inplace = True)
df_data.drop(axis=1, columns = ['Date'], inplace=True)

In [15]:
df_data_features = df_data.iloc[:-1 , :]
df_data_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
499,132.160004,132.429993,130.779999,130.960007,88223700
500,131.320007,133.460007,131.100006,131.970001,54930100
501,133.990005,137.339996,133.509995,136.690002,124486200
502,138.050003,138.789993,134.339996,134.869995,121047300


In [16]:
df_data_targets = df_data.iloc[1:, 0].rename ("Targets")
df_data_targets

1       35.994999
2       36.132500
3       37.174999
4       37.389999
5       37.822498
          ...    
500    131.320007
501    133.990005
502    138.050003
503    135.580002
504    134.080002
Name: Targets, Length: 504, dtype: float64

In [18]:
df_data_features['Target'] = list(df_data_targets)

first_column = df_data_features.pop('Target')

df_data_features.insert(0,'Target', first_column)
df_data_final = df_data_features

df_data_final

/tmp/ipykernel_8787/3433040288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_targets)


,Target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
499,131.320007,132.160004,132.429993,130.779999,130.960007,88223700
500,133.990005,131.320007,133.460007,131.100006,131.970001,54930100
501,138.050003,133.990005,137.339996,133.509995,136.690002,124486200
502,135.580002,138.050003,138.789993,134.339996,134.869995,121047300


## Train-Test Split

In [23]:
import numpy as пр
                
df_randomized = df_data_final.sample(frac=1,random_state=111)
df_randomized


,Target,Open,High,Low,Close,Volume
404,111.970001,112.599998,113.775002,110.000000,112.727501,212403600
181,54.737499,55.345001,55.639999,54.367500,54.432499,221652400
434,111.620003,112.680000,112.860001,109.160004,111.809998,183055400
331,71.269997,70.449997,71.135002,69.987503,70.792503,117087600
183,54.637501,55.257500,55.622501,54.297501,54.419998,124763200
...,...,...,...,...,...,...
86,50.474998,51.470001,51.855000,50.207500,50.715000,155054800
212,64.262497,64.332497,64.462502,63.845001,64.375000,103272000
468,115.550003,120.500000,121.989998,116.050003,116.320000,154515300
364,83.312500,86.180000,86.949997,83.555000,84.699997,200146000


In [29]:
train_data, test_data = df_randomized[:int(0.8*len(df_randomized))], df_randomized[int(0.8*len(df_randomized)):]
print(train_data.shape,test_data.shape)

(403, 6) (101, 6)


## Set path and upload data to S3

In [32]:
import os
prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)


s3://stockpricemarketprediction/xgboost-as-a-built-in-algo/train/train.csv
s3://stockpricemarketprediction/xgboost-as-a-built-in-algo/test/test.csv


In [33]:
train_data.to_csv(train_csv_path,index=False,header=False)
test_data.to_csv(test_csv_path,index=False,header=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


## Building XGBoost

In [35]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

### Find XGBoost Image URI and Binds and XGBoost Container

In [ ]:
xgboost_container = image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-2")
display(xgboost_container)

#### Initializing Hyperparameters 

In [38]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "verbosity":"1",
        "objective":"reg:squarederror",
        "early_stopping_rounds":10,
        "num_round":300}

In [ ]:
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix, 'output')
print(output_path)

### Constructing a SageMaker estimator that calls the xgboost-container

In [40]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),#IAMROLE
                                          instance_count=1, 
                                          instance_type='ml.m4.xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                         use_spot_instances=True,
                                         max_run=300,
                                         max_wait=600)

Providing Paths to Training and Validation Datasets

In [41]:
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format (bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)


In [44]:
train_input

### Training 

In [45]:
estimator.fit({'train': train_input, 'validation' : test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-02-03-21-48-08-805


2024-02-03 21:48:08 Starting - Starting the training job...
2024-02-03 21:48:25 Starting - Preparing the instances for training.........
2024-02-03 21:49:46 Downloading - Downloading input data...
2024-02-03 21:50:16 Downloading - Downloading the training image...
2024-02-03 21:51:07 Training - Training image download completed. Training in progress....
2024-02-03 21:51:27 Uploading - Uploading generated training model[2024-02-03 21:51:23.174 ip-10-2-228-177.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-02-03:21:51:23:INFO] Imported framework sagemaker_xgboost_container.training
[2024-02-03:21:51:23:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-02-03:21:51:23:INFO] No GPUs detected (normal if no gpus installed)
[2024-02-03:21:51:23:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-02-03:21:51:23:INFO] Determined delimiter of CSV input is ','
[2024-02-03:21:51:23:INFO] Determined delimit

### Deploy Trained XGB Model as EndPoint

We will be invoking the endpoint using API , we can also do it with S3. Our datatype will be JSON , For S3 we can use CSV.

In [47]:
from sagemaker. serializers import CSVSerializer
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-02-04-00-31-20-989
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-02-04-00-31-20-989
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-02-04-00-31-20-989


------!

In [ ]:
xgb_predictor.endpoint_name

## Make Predictions Using Endpoint

In [67]:
# initialize parameters
start_date = datetime(2021, 1, 4)
end_date = datetime(2021, 1, 5)
# get the data
df_data = yf.download('AAPL', start = start_date, end = end_date)
df_data.reset_index(inplace=True)
df_data


[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,127.164146,143301900


In [68]:
df_data.drop(axis=1, columns = ['Adj Close'], inplace=True)
df_data.drop(axis=1, columns = ['Date'], inplace=True)
data_features_array = df_data.values
data_features_array


array([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]])

## Serialize Data

### Inference - Serialized Input By Sagemaker Function

In [69]:
Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
print(Y_pred_Fcn,type(Y_pred_Fcn))

129.8251495361328
 <class 'str'>


### Inference - Serialized Input By Built-in Function (Lamda Function Friendly)

In [70]:
Input=[[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08],[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08], 1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]]

Serialized_Input = ','.join(map(str, Input[0]))
print(Serialized_Input,type(Serialized_Input))

133.520004,133.610001,126.760002,129.410004,143301900.0 <class 'str'>


In [71]:
Y_pred= xgb_predictor.predict(Serialized_Input).decode('utf-8')
Y_pred

'129.8251495361328\n'

# Lamda Function Handler

In [82]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-02-04-00-31-20-989'
runtime = boto3.client('runtime.sagemaker')
email_client = boto3.client('sns')

def lambda_handler(event, context):
    inputs = event['data']
    result = []
    
    for input in inputs:
        serialized_input = ','.join(map(str, input))
        response = runtime. invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                 ContentType= 'text/csv',
                                 Body=serialized_input)
    
        result.append(response['Body'].read().decode())
    
#         This Block is used for Sending the Email
    response_sns = email_client.publish(
        TopicArn = 'Copy-TopicArn-From-SNS-TopicArn',
        Message = 'Prediction is '+ str(result),
        Subject = 'EMC Finance - Daily Prediction')
    
    return result

In [85]:
Input_json = {
  "data": [
    [
      133.520004,
      133.610001,
      126.760002,
      129.410004,
      143301900
    ],
    [
      133.520004,
      133.610001,
      126.760002,
      129.410004,
      143301900
    ],
    [
      133.520004,
      133.610001,
      126.760002,
      129.410004,
      143301900
    ]
  ]
}
result = lambda_handler(Input_json,__)
result

['129.8251495361328\n', '129.8251495361328\n', '129.8251495361328\n']

In [88]:
#importing the requests library
import requests
# defining the api-endpoint
API_ENDPOINT = "https://6########5.execute-api.us-east-1.amazonaws.com/XGBoost"
# data to be sent to api
json = {
    "data": [
    [
      133.520004,
      133.610001,
      126.760002,
      129.410004,
      143301900
    ]
  ]
}

r = requests.post(url = API_ENDPOINT, json=json)

In [87]:
print(f"Status code: {r.status_code}, Response: {r.json}")

Status code: 200, Response: <bound method Response.json of <Response [200]>>


## Close and Terminate

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()